In [1]:
import numpy as np
from matplotlib import pyplot as plt
import os
from keras.api.models import Model
from keras.api.layers import Input, Dense, Activation, Flatten
from keras.api.optimizers import Adam
from keras.api.losses import binary_crossentropy
import random

In [2]:

dataset_toystory_path = './datasets/toy_story_1_01'
dataset_ernest_path = './datasets/ernest_celestine_01'
datasetnpy_path = './datasets/dataset.npy'

dataset = []
for img in os.listdir(dataset_ernest_path):
    dataset.append(plt.imread(os.path.join(dataset_ernest_path, img)))


for img in os.listdir(dataset_toystory_path):
    dataset.append(plt.imread(os.path.join(dataset_toystory_path, img)))

dataset = np.array(dataset)

with open(datasetnpy_path, 'wb') as f:
    np.save(f, dataset)

In [3]:
y_arr = np.zeros(1000)
y_arr[500:] = 1
y_arr = y_arr.reshape((-1,1))
print(y_arr)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.

In [4]:
with open(datasetnpy_path, 'rb') as f:
    dataset = np.load(f)

In [6]:
print(f'dataset shape : {dataset.shape}')

dataset shape : (1000, 540, 920, 3)


In [7]:
def compute_mean(image):
    image_size = image.shape[0] * image.shape[1] * image.shape[2]
    
    return (np.sum(image.flatten())/image_size)


In [8]:
carac_vector = np.zeros(len(dataset))
for i, image in enumerate(dataset):
    carac_vector[i] = compute_mean(image)

carac_vector = carac_vector.reshape((1,-1))
print(f'carac vector shape : {carac_vector.shape}')

carac vector shape : (1, 1000)


In [9]:
y_tab = np.random.choice([0,1], size=(1000,))
print(y_tab.shape)

(1000,)


In [10]:
input = Input(shape=(540,920,3))
x = Flatten()(input)
x = Dense(units=32, activation='relu')(x)
x = Dense(units=16, activation='relu')(x)
output = Dense(units=1, activation='sigmoid')(x)

model = Model(input, output)

In [11]:
model.summary()
optimizer = Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss=binary_crossentropy, metrics=['accuracy'])

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 540, 920, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1490400)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │    47,692,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,693,377 (181.94 MB)

 Trainable params: 47,693,377 (181.94 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.fit(dataset, y_arr, epochs=5, batch_size=16, shuffle=True)

Epoch 1/5
18/63 ━━━━━━━━━━━━━━━━━━━━ 18s 415ms/step - accuracy: 0.4919 - loss: 332.4404

KeyboardInterrupt: 

In [95]:
model.evaluate(dataset)